# X (Twitter) Sentiment Analysis Model using Machine Learning

Loading Dataset

In [1]:
# Configure the path of Kaggle.json file
!mkdir -p ~/.kaggle

!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [2]:
#API to fetch dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 77% 62.0M/80.9M [00:00<00:00, 182MB/s]
100% 80.9M/80.9M [00:00<00:00, 174MB/s]


In [3]:
# Extracting the compressed dataset
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


Importing the Dependencies

In [4]:
import numpy as np # For Mathematical Calculations
import pandas as pd # For dataframe processing & analysis
import re # Regular Expression for patterns
from nltk.corpus import stopwords # Natural Language Toolkit processing
from nltk.stem.porter import PorterStemmer # Stemming
from sklearn.feature_extraction.text import TfidfVectorizer # Text Vectorization
from sklearn.model_selection import train_test_split # For splitting data
from sklearn.linear_model import LogisticRegression # For Logistic Regression
from sklearn.metrics import accuracy_score # For Accuracy Score

In [5]:
import nltk
nltk.download('stopwords')
# stopwords are the enlgish used commonly in conversations but have not much significant importance here


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
# Printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Processing

In [7]:
# Loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1') # encoding is done with industry standards

In [8]:
# Checking the number of rows and columns
twitter_data.shape

(1599999, 6)

In [9]:
# Printing the first 5 rows of the dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [10]:
# Naming the columns & reading the dataset again

column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names = column_names, encoding='ISO-8859-1') # Changed the column names here

In [11]:
# Checking the number of rows & cols
twitter_data.shape

(1600000, 6)

In [12]:
# Printing the first 5 rows of the dataframe
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [13]:
# Counting the number of missing values in the dataset
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [14]:
# Checking the distribution of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


Convert the target "4" to "1"

In [15]:
twitter_data.replace({'target' : {4:1}}, inplace = True)

In [16]:
# Checking the distribution of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


0 --> Negative Tweet

1 --> Positive Tweet

**Stemming**


Stemming is the process of reducing a word to its Root word

example: actor, actress, acting = act


In [17]:
# Reduce every word to its root word
port_stem  = PorterStemmer()

In [18]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Initialize stopwords
stop_words = set(stopwords.words('english'))

def stemming(content):
    try:
        port_stem = PorterStemmer()  # Instantiate inside the function
        stemmed_content = re.sub('[^a-zA-Z]', ' ', content).lower()
        return ' '.join(port_stem.stem(word) for word in stemmed_content.split() if word not in stop_words)
    except Exception as e:
        print(f"Error processing content: {content}. Error: {e}")
        return ""  # Return an empty string on error

def process_data(df):
    with ThreadPoolExecutor() as executor:
        return list(tqdm(executor.map(stemming, df['text']), total=len(df)))

# Process the DataFrame in chunks
chunk_size = 50000  # Adjust based on your memory capacity
num_chunks = len(twitter_data) // chunk_size + 1
stemmed_contents = []

for i in tqdm(range(num_chunks)):
    start = i * chunk_size
    end = min((i + 1) * chunk_size, len(twitter_data))
    chunk = twitter_data.iloc[start:end]
    stemmed_chunk = process_data(chunk)
    stemmed_contents.extend(stemmed_chunk)

# Add the stemmed content back to the DataFrame
twitter_data['stemmed_content'] = stemmed_contents

 97%|█████████▋| 32/33 [07:14<00:13, 13.56s/it]
0it [00:00, ?it/s]
100%|██████████| 33/33 [07:14<00:00, 13.17s/it]


In [19]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [20]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


 Splitting the data to training data & test data

In [21]:
# Separating the data and target after stemming
X = twitter_data['stemmed_content']
Y = twitter_data['target']

# Splitting the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [22]:
# Vectorization: Fit on TRAINING DATA only to prevent data leakage
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Save the vectorizer
import pickle
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

**Training the Machine Learning Model**

Logisitc Regression

In [23]:
# Model Training
model = LogisticRegression(max_iter=1000)
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

**Model Evaluation**

Accuracy Score

In [24]:
X_train_pred = model.predict(X_train)
training_accuracy = accuracy_score(Y_train, X_train_pred)
print("Training Accuracy:", training_accuracy)

Training Accuracy: 0.79871953125


In [25]:
# Test accuracy
X_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(Y_test, X_test_pred)
print("Test Accuracy:", test_accuracy)

Test Accuracy: 0.77668125


**Model Accuracy = 77.6 %**

Saving the trained Model

In [26]:
import pickle

In [27]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb')) # wb -> here stands for writing on file

Using the saved model for future predicitons

In [28]:
# Loading the saved model
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb')) # rb -> here stands for writing on file

In [29]:
X_new = X_test[100]
prediction = loaded_model.predict(X_new)
print('Expected : ',Y_test[100] )
print('Predicted : ',prediction)

if(prediction[0]==0):
  print('\nNegative Tweet')
else:
  print('\nPositive Tweet')

Expected :  0
Predicted :  [0]

Negative Tweet


In [30]:
X_new = X_test[1]
prediction = loaded_model.predict(X_new)
print('Expected : ',Y_test[1] )
print('Predicted : ',prediction)

if(prediction[0]==0):
  print('\nNegative Tweet')
else:
  print('\nPositive Tweet')

Expected :  0
Predicted :  [1]

Positive Tweet


Now a custom Tweet Sentiment Analysis

In [ ]:
import pickle
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Load the model and vectorizer (ensure they are saved during training)
loaded_model = pickle.load(open('trained_model.sav', 'rb'))
vectorizer = pickle.load(open('vectorizer.pkl', 'rb'))  # Save this during training

def preprocess_tweet(tweet):
    # Stemming and cleaning
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    processed = re.sub('[^a-zA-Z]', ' ', tweet).lower().split()
    processed = [ps.stem(word) for word in processed if word not in stop_words]
    return ' '.join(processed)

# Input a tweet
tweet = input("Enter a tweet: ")

# Preprocess and vectorize
processed_tweet = preprocess_tweet(tweet)
X_new = vectorizer.transform([processed_tweet])

# Predict
prediction = loaded_model.predict(X_new)[0]

# Map prediction to label
if prediction == 0:
    print("Prediction: Negative")
elif prediction == 1:
    print("Prediction: Positive")
else:
    print("Prediction: Neutral")  # Not supported by the model (included for completeness)